# **Cumpli-Miento: Detección de cumplimiento del RGPD en políticas de privacidad web**

- Autores: Daniel Carreres y Juan Sanz
- Fecha: 27/07/2024
- Versión: 1.0

# Tabla de contenidos (índice)

>[Cumpli-Miento: Detección de cumplimiento del RGPD en políticas de privacidad web](#scrollTo=xaAXRslVRqWQ)

>[Tabla de contenidos (índice)](#scrollTo=xaAXRslVRqWQ)

>>[0. Cargar los datos](#scrollTo=qgwDlksWG55_)

>>[1. Preparación de los Datos](#scrollTo=EyJFhIcuHpab)

>>[2. Definición y Entrenamiento de Modelos](#scrollTo=U-OvYGEiG9kE)

>>[3. Evaluación de Políticas de Privacidad](#scrollTo=mLk5ZVbTH6qW)



## 0. Cargar los datos

In [ ]:
import zipfile
import os

# Definir la ruta del archivo zip cargado
zip_file_path = '/content/privacy_policys (4).zip'  # Cambia esta ruta por la ruta correcta de tu archivo zip
extract_path = '/content/privacy_policys'  # Cambia esta ruta por la ruta donde quieres descomprimir los archivos

# Descomprimir el archivo zip
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Listar los archivos extraídos para asegurarse de que se han descomprimido correctamente
extracted_files = os.listdir(extract_path)
print("Archivos extraídos:", extracted_files[:10])  # Muestra solo los primeros 10 archivos para brevedad


Archivos extraídos: ['privacy_policys (2)']


## 1. Preparación de los Datos

In [ ]:
import os
import pandas as pd

# Cargar el archivo CSV
csv_file_path = '/content/Refined_Corrected_Data.csv'
data = pd.read_csv(csv_file_path)

# Verificar los archivos en el directorio
extract_path = '/content/privacy_policys/privacy_policys (2)'
renamed_files = os.listdir(extract_path)
print("Archivos renombrados:", renamed_files[:10])

# Añadir una nueva columna 'Filename' en el DataFrame con los nombres simples de las empresas
data['Filename'] = data['Empresa'].apply(lambda x: x + '.html')

# Verificar las primeras filas del DataFrame
print(data.head())


## 2. Definición y Entrenamiento de Modelos

In [ ]:
# Definir la función para extraer texto de archivos HTML
def extract_text_from_html(file_path):
    if file_path and os.path.isfile(file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
            soup = BeautifulSoup(content, 'html.parser')
            text = soup.get_text(separator=' ', strip=True)
        return text
    return None

# Aplicar la función refinada al DataFrame
def match_and_extract_text_using_filenames(df, extract_path):
    df['Texto'] = df['Filename'].apply(lambda filename: extract_text_from_html(os.path.join(extract_path, filename)))
    return df

# Aplicar la función para extraer texto al DataFrame
data_with_text = match_and_extract_text_using_filenames(data, extract_path)

# Filtrar las filas donde el texto fue extraído correctamente
data_with_text = data_with_text.dropna(subset=['Texto'])

# Mostrar el DataFrame con el texto incluido
print(data_with_text.head())


In [ ]:
# Verificar las columnas del DataFrame
print("Columnas del DataFrame:", data.columns)

# Añadir nuevamente la columna 'Filename' en el DataFrame
data['Filename'] = data['Empresa'].apply(lambda x: x.lower() + '.html')

# Verificar las primeras filas del DataFrame con la nueva columna
print(data.head())

# Aplicar la función refinada al DataFrame para extraer texto
def match_and_extract_text_using_filenames(df, extract_path):
    df['Texto'] = df['Filename'].apply(lambda filename: extract_text_from_html(os.path.join(extract_path, filename)))
    return df

# Aplicar la función para extraer texto al DataFrame
data_with_text = match_and_extract_text_using_filenames(data, extract_path)

# Filtrar las filas donde el texto fue extraído correctamente
data_with_text = data_with_text.dropna(subset=['Texto'])

# Mostrar el DataFrame con el texto incluido
data_with_text.head()


## 3. Evaluación de Políticas de Privacidad

In [ ]:
import requests
from bs4 import BeautifulSoup

# Función para extraer texto de una URL
def extract_text_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator=' ', strip=True)
        return text
    else:
        print(f"Error al acceder a la URL: {response.status_code}")
        return None


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Derechos a evaluar
derechos = ['Derecho a ser informado', 'Derecho de acceso', 'Derecho de rectificación',
            'Derecho de eliminación', 'Derecho de procesamiento restringido',
            'Derecho a la portabilidad de los datos', 'Derecho de oposición',
            'Derechos relacionados con la toma de decisiones y la elaboración de perfiles automatizados']

# Diccionario para almacenar modelos y vectorizadores para cada derecho
models = {}
vectorizers = {}

# Entrenar un modelo para cada derecho
for derecho in derechos:
    X = data_with_text['Texto']
    y = data_with_text[derecho]

    # Verificar que haya al menos dos clases en los datos
    if len(y.unique()) > 1:
        # Crear un pipeline de TF-IDF Vectorizer y Logistic Regression
        pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression(max_iter=1000))

        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Entrenar el modelo
        pipeline.fit(X_train, y_train)

        # Evaluar el modelo
        accuracy = pipeline.score(X_test, y_test)
        print(f"Accuracy for {derecho}: {accuracy}")

        # Guardar el modelo y el vectorizador
        models[derecho] = pipeline.named_steps['logisticregression']
        vectorizers[derecho] = pipeline.named_steps['tfidfvectorizer']
    else:
        print(f"No hay suficientes datos para entrenar el modelo para {derecho}")


No hay suficientes datos para entrenar el modelo para Derecho a ser informado
Accuracy for Derecho de acceso: 1.0
Accuracy for Derecho de rectificación: 1.0
Accuracy for Derecho de eliminación: 1.0
Accuracy for Derecho de procesamiento restringido: 0.7777777777777778
Accuracy for Derecho a la portabilidad de los datos: 1.0
Accuracy for Derecho de oposición: 1.0
Accuracy for Derechos relacionados con la toma de decisiones y la elaboración de perfiles automatizados: 0.5555555555555556


In [ ]:
import requests
from bs4 import BeautifulSoup

# Función para extraer texto de una URL
def extract_text_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        text = soup.get_text(separator=' ', strip=True)
        return text
    else:
        print(f"Error al acceder a la URL: {response.status_code}")
        return None


In [ ]:
# Función para evaluar el texto extraído para cada derecho
def evaluate_rights_from_url(url, models, vectorizers):
    text = extract_text_from_url(url)
    if text:
        results = {}
        for derecho in derechos:
            if derecho in models:
                vectorizer = vectorizers[derecho]
                model = models[derecho]
                X_new = vectorizer.transform([text])
                prediction = model.predict(X_new)
                results[derecho] = 'Cumple' if prediction[0] else 'No cumple'
            else:
                results[derecho] = 'Sin datos suficientes'
        return results
    else:
        return None

# Ejemplo de URL para evaluar
url_to_evaluate = "https://pokedoku.com/policy"
results = evaluate_rights_from_url(url_to_evaluate, models, vectorizers)
print("Resultados de la evaluación:")
for derecho, resultado in results.items():
    print(f"{derecho}: {resultado}")


Resultados de la evaluación:
Derecho a ser informado: Sin datos suficientes
Derecho de acceso: Cumple
Derecho de rectificación: Cumple
Derecho de eliminación: Cumple
Derecho de procesamiento restringido: Cumple
Derecho a la portabilidad de los datos: Cumple
Derecho de oposición: Cumple
Derechos relacionados con la toma de decisiones y la elaboración de perfiles automatizados: Cumple
